<a href="https://colab.research.google.com/github/jimin0/Graduation_Project_HE_Study/blob/main/ex5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 졸업작품
>SEAL-Python 정리

[TODO]


1.   암호문 type 확인
2.   암호문 + 암호문
3.   암호문 + 비암호문
4.   암호문 * 암호문
5.   암호문 * 비암호문
6.   복호화
7.   재선형화?


## 환경설정

In [1]:
!git clone https://github.com/Huelse/SEAL-Python.git
%cd SEAL-Python

# Install dependencies
!pip3 install numpy pybind11

# Init the SEAL and pybind11
!git submodule update --init --recursive
# Get the newest repositories (dev only)
# git submodule update --remote

# Build the SEAL lib
%cd SEAL
!cmake -S . -B build -DSEAL_USE_MSGSL=OFF -DSEAL_USE_ZLIB=OFF
!cmake --build build
%cd ..

# Run the setup.py
!python3 setup.py build_ext -i


Cloning into 'SEAL-Python'...
remote: Enumerating objects: 1628, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1628 (delta 176), reused 191 (delta 162), pack-reused 1376
Receiving objects: 100% (1628/1628), 8.68 MiB | 17.77 MiB/s, done.
Resolving deltas: 100% (882/882), done.
/content/SEAL-Python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 3.6 MB/s eta 0:00:00
Submodule 'SEAL' (https://github.com/microsoft/SEAL.git) registered for path 'SEAL'
Submodule 'pybind11' (https://github.com/pybind/pybind11.git) registered for path 'pybind11'
Cloning into '/content/SEAL-Python/SEAL'...
Cloning into '/content/SEAL-Python/pybind11'...
Submodule path 'SEAL': checked out '206648d0e4634e5c61dcf9370676630268290b59'
Submodule path 'pybind11': checked out 'be97c5a98b4b252c524566f508b5c79410d118c6'
/content/SEAL-Python/SEAL
-- Build type (CMAKE_BUILD_TYPE): Release
-- The CXX compiler identification is GNU 11.4.0
-- 

## 동형암호

### bgv 객체 생성 및 키 생성

In [2]:
# SEAL 라이브러리의 모든 클래스와 함수를 임포트합니다.
from seal import *
import numpy as np

# BGV 암호화 스킴을 사용할 암호화 매개변수 객체를 생성합니다.
parms = EncryptionParameters(scheme_type.bgv)

"""
poly_modulus_degree가 클수록 암호화된 텍스트의 크기가 커지고 모든 연산이 느려지지만,
더 복잡한 암호화된 계산을 가능하게 함. 권장되는 값들은 1024, 2048, 4096, 8192, 16384, 32768이지만,
이 범위를 넘어가는 것도 가능
"""
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(8192)

context = SEALContext(parms)

# 키 생성

keygen = KeyGenerator(context)

secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
galois_keys =  keygen.create_galois_keys()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

### [TODO]

#### 1.암호문 type 확인

In [3]:
# 암호화 및 복호화
plaintext = Plaintext("1000")
print("평문화된 텍스트 값 (문자열):", plaintext.to_string())
print("평문화된 텍스트 객체 주소:", plaintext)
print("평문화된 텍스트 값의 타입:", type(plaintext.to_string()))
print("평문화된 텍스트 객체의 타입:", type(plaintext))

print()
ciphertext = Ciphertext()
ciphertext = encryptor.encrypt(plaintext)
print("암호화된 텍스트 값 (문자열):", ciphertext.to_string())
print("암호화된 텍스트 객체 주소:", ciphertext)
print("암호화된 텍스트 값의 타입:", type(ciphertext.to_string()))
print("암호화된 객체의 타입:", type(ciphertext))

print()
decrypted = Plaintext()
decryptor.decrypt(ciphertext, decrypted)
print("복호화된 텍스트 값 (문자열):", decrypted.to_string())
print("복호화된 텍스트 객체 주소:", decrypted)
print("복호화된 텍스트 값의 타입:", type(decrypted.to_string()))
print("복호화된 객체의 타입:", type(decrypted))


평문화된 텍스트 값 (문자열): 1000
평문화된 텍스트 객체 주소: <seal.Plaintext object at 0x78de196402f0>
평문화된 텍스트 값의 타입: <class 'str'>
평문화된 텍스트 객체의 타입: <class 'seal.Plaintext'>

암호화된 텍스트 값 (문자열): b'^\xa1\x10\x04\x01\x02\x00\x00\x90\x99\x06\x00\x00\x00\x00\x00(\xb5/\xfd\xa0a\x00\x08\x00\\\x15\rn\xfd\xdf\xa5h)\x10\xb0\x1d\xe6\xa9\xaf\x8c)\x8e\xcf\xf9\xbc\xe0(\xf9\\\xe5\x03\xces\xaf\xb9\xa5\x02\xf8\x90@\xb3_\x9a\xa0\x03\x134I\x0c)\xa5\x94R\xa6\x96h\x91h\x98h\xb7\x9c\x114\xb0p\xf0\xab\x9e\x81\x0cg\xdcwK\xd4\x98\x12"\xf3\xd7\x17\x1c\xc6\xbc\xe9\xf1\xa9\xdd\x1aI>\xd4O%k\xf0 \xe3\xe3\xdb\xb57\x9aS\xf2|\xfb\xa0\xee\xf2\x84\xda\xafo\xf0X\xe9\x83\xf8)\x8c\x15\x99\x07\xb2>\x1d\xed;\x91\x85\xc2\xa7&\xcerS\xe8\xdd;\x1d\xc4\xc6\x08=Ki\xb3\xef\xb3\xd2\xa7\\\xba\xd8\xf6\xc4\xf8\xd4\x80`\nAI^e\x13;0\x97\xe8\xd9.-\x0c\x02x\xcf>\x99\xc4\xc9\r\xda\xb7\x89\x1e\xc7\xc24\x7f\x0b\x8dp,~\xfd\x10\x8e\x94\n{\xfb\xbf\x84Vf\x16\xf8\xf70\xb0\xd2\xa4\x1b\x7f\xa3\x16\xb5\x10\x1d\xfe\x85"\xa9\x0f\xa4\xc9\x7fK#\xdf\x02\xcdg\xd3\xf0\x9a\xa8\x0

#### 2.암호문 + 암호문
>모듈 2가지
1.   evaluator.add(ciphertext1,ciphertext2) : return 값 있음.
2.   evaluator.add_inaplace(ciphertext1,ciphertext2) : ciphertext+=ciphertext2 (no return)


##### 전체 예제 코드

In [12]:
# 암호화
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")

ciphertext1 = encryptor.encrypt(plaintext1)       # 6를 암호화하여 ciphertext1에 저장
ciphertext2 = encryptor.encrypt(plaintext2)       # 3을 암호화하여 ciphertext2에 저장

# 암호문 + 암호문 덧셈 1 (add 모듈)
result_ciphertext = evaluator.add(ciphertext1, ciphertext2)

# 암호문 + 암호문 덧셈 2 (add_inplace 모듈)
evaluator.add_inplace(ciphertext1, ciphertext2)

# 결과 복호화
# result_ciphertext를 복호화하여 그 결과를 decrypted에 저장
decrypted = Plaintext()
decryptor.decrypt(result_ciphertext, decrypted)

print("결과:", decrypted)     # to_string 안할시, 복호화된 배열 주소가 출력됨.
decrypted_result =  decrypted.to_string()
print("결과:",type(decrypted_result))
print("결과:",decrypted_result)

# 문자열 형식을 기반으로 진수 판단
# SEAL은 일반적으로 16진수 형태의 문자열을 반환합니다.
if all(c in '0123456789ABCDEF' for c in decrypted_result.upper()):
    print("복호화된 결과는 16진수를 나타냅니다.")
else:
    print("복호화된 결과의 진수를 판단할 수 없습니다.")


결과: <seal.Plaintext object at 0x78de19775df0>
결과: <class 'str'>
결과: 9
복호화된 결과는 16진수를 나타냅니다.


##### 코드 요약

In [9]:
# 암호화 초기 단계 : plalintext 객체 생성
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")


# Chiphertext
ciphertext1 = encryptor.encrypt(plaintext1)
ciphertext2 = encryptor.encrypt(plaintext2)


# 암호문 + 비암호문 덧셈 (add module 사용)
result_ciphertext = evaluator.add(ciphertext1, ciphertext2)

# 결과 복호화
decrypted = Plaintext()
decryptor.decrypt(result_ciphertext, decrypted) # 인자 type : (ciphertext, plaintext)
print("결과:", decrypted)                        # to_string 안할시, 배열 주소가 출력됨.
print("결과:", decrypted.to_string())

noise_budget_after = decryptor.invariant_noise_budget(result_ciphertext)
print("덧셈 후 노이즈 예산:", noise_budget_after, "bits")

결과: <seal.Plaintext object at 0x78de1881eb70>
결과: 9
덧셈 후 노이즈 예산: 152 bits


#### 3.비암호문 + 암호문
>모듈 1가지
1.   evaluator.add_plain(ciphertext1,ciphertext2) : return 값 있음.

In [17]:
# 암호화 초기 단계 : plalintext 객체 생성
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")


# Chiphertext
ciphertext1 = encryptor.encrypt(plaintext1)

# 암호문 + 비암호문 덧셈 (add_plain 모듈 사용)
result_ciphertext = evaluator.add_plain(ciphertext1, plaintext2)

# 결과 복호화
decrypted = Plaintext()
decryptor.decrypt(result_ciphertext, decrypted) # 인자 type : (ciphertext, plaintext)
print("결과:", decrypted)                         # to_string 안할시, 배열 주소가 출력됨.
print("결과:", decrypted.to_string())

noise_budget_after = decryptor.invariant_noise_budget(result_ciphertext)
print("덧셈 후 노이즈 예산:", noise_budget_after, "bits")

결과: <seal.Plaintext object at 0x78de18827ef0>
결과: 9
덧셈 후 노이즈 예산: 152 bits


####4.암호문 * 암호문
>모듈 2가지
1.   evaluator.multiply(ciphertext1,ciphertext2) : return 값 있음.
2.   evaluator.multiply_inaplace(ciphertext1,ciphertext2) : ciphertext+=ciphertext2 (no return)

In [25]:
# plaintext 객체 생성
plaintext1 = Plaintext("6")
plaintext2 = Plaintext("3")

# 암호화
ciphertext1 = encryptor.encrypt(plaintext1)
ciphertext2 = encryptor.encrypt(plaintext2)

# 암호문 곱셈 (두가지 방법중 택 1)
#result = evaluator.multiply(ciphertext1, ciphertext2)
evaluator.multiply_inplace(ciphertext1, ciphertext2)

# 결과 복호화
decrypted = Plaintext()
decryptor.decrypt(ciphertext1, decrypted)
print("multiply 모듈 쓴 결과(16진수):", decrypted.to_string())            # 결과 12 이지만, 16진수 12 = 10진수 18임
print("multiply 모듈 쓴 결과(10진수):", int(decrypted.to_string(),16))    # 6진수라서 to_String한 결과는 16진수 형태라서 10진수로 바꿈.

# 곱셈 후 노이즈 예산 확인
noise_budget_after = decryptor.invariant_noise_budget(ciphertext1)
print("곱셈 후 노이즈 예산:", noise_budget_after, "bits")


multiply 모듈 쓴 결과(16진수): 12
multiply 모듈 쓴 결과(10진수): 18
곱셈 후 노이즈 예산: 124 bits


####5.비암호문*비암호문

####6.복호화